In [18]:
import pandas as pd
import numpy as np

In [19]:
listing_details = pd.read_csv('output/listing_details.csv')
listing_details.drop(['latitude', 'longitude', 'image_urls'], axis=1, inplace=True)

In [20]:
listing_details.shape[0]

32461

In [21]:
listing_details_cleaned = pd.read_csv('output/listing_details_cleaned.csv')
listing_details_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18534 entries, 0 to 18533
Data columns (total 29 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Tỉnh/Thành phố                        18534 non-null  object 
 1   Thành phố/Quận/Huyện/Thị xã           18534 non-null  object 
 2   Xã/Phường/Thị trấn                    18321 non-null  object 
 3   Đường phố                             15355 non-null  object 
 4   Chi tiết                              18534 non-null  object 
 5   Nguồn thông tin                       18534 non-null  object 
 6   Tình trạng giao dịch                  18534 non-null  object 
 7   Thời điểm giao dịch/rao bán           18524 non-null  object 
 8   Thông tin liên hệ                     0 non-null      float64
 9   Giá rao bán/giao dịch                 17012 non-null  float64
 10  Loại đơn giá (đ/m2 hoặc đ/m ngang)    18534 non-null  object 
 11  Số tầng công tr

In [22]:
listing_details_cleaned.rename(columns={'Nguồn thông tin': 'url'}, inplace=True)
listing_details_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18534 entries, 0 to 18533
Data columns (total 29 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Tỉnh/Thành phố                        18534 non-null  object 
 1   Thành phố/Quận/Huyện/Thị xã           18534 non-null  object 
 2   Xã/Phường/Thị trấn                    18321 non-null  object 
 3   Đường phố                             15355 non-null  object 
 4   Chi tiết                              18534 non-null  object 
 5   url                                   18534 non-null  object 
 6   Tình trạng giao dịch                  18534 non-null  object 
 7   Thời điểm giao dịch/rao bán           18524 non-null  object 
 8   Thông tin liên hệ                     0 non-null      float64
 9   Giá rao bán/giao dịch                 17012 non-null  float64
 10  Loại đơn giá (đ/m2 hoặc đ/m ngang)    18534 non-null  object 
 11  Số tầng công tr

In [23]:
cleaned = pd.merge(listing_details_cleaned, listing_details, how='left', on='url')
cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18536 entries, 0 to 18535
Data columns (total 35 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Tỉnh/Thành phố                        18536 non-null  object 
 1   Thành phố/Quận/Huyện/Thị xã           18536 non-null  object 
 2   Xã/Phường/Thị trấn                    18323 non-null  object 
 3   Đường phố                             15357 non-null  object 
 4   Chi tiết                              18536 non-null  object 
 5   url                                   18536 non-null  object 
 6   Tình trạng giao dịch                  18536 non-null  object 
 7   Thời điểm giao dịch/rao bán           18526 non-null  object 
 8   Thông tin liên hệ                     0 non-null      float64
 9   Giá rao bán/giao dịch                 17014 non-null  float64
 10  Loại đơn giá (đ/m2 hoặc đ/m ngang)    18536 non-null  object 
 11  Số tầng công tr

## Columns to fix: 
Xã/Phường/Thị trấn, Đường phố, Giá rao bán/giao dịch, Số tầng công trình, Tổng diện tích sàn, Đơn giá xây dựng, Chất lượng còn lại, Diện tích đất (m2), Kích thước mặt tiền (m), Kích thước chiều dài (m), Số mặt tiền tiếp giáp, Hình dạng, Độ rộng ngõ/ngách nhỏ nhất (m), Khoảng cách tới trục đường chính (m)

In [24]:
import numpy as np

def correct_phuong(row):
    xaphuong = row['Xã/Phường/Thị trấn']
    short_add = row['short_address']
    if type(xaphuong) == float or type(short_add) == float:
        return 0
    if xaphuong in short_add:
        return 1
    return 0

cleaned['correct_phuong'] = cleaned.apply(correct_phuong, axis=1)
incorrect_phuong = cleaned[cleaned['correct_phuong'] == 0]

In [25]:
pd.reset_option('display.max_colwidth', None)
pd.reset_option('display.max_rows', None)

incorrect_phuong[['Xã/Phường/Thị trấn', 'short_address']]

,Xã/Phường/Thị trấn,short_address
176,NaN,Nhà 2 mặt tiền hẻm xe tải thông hướng Âu Cơ Tr...
305,NaN,Nhà 3 tầng Nguyễn Viết Xuân Hòa Minh Liên Chiể...
360,Phường Yên Nghĩa,"Yên Nghĩa, Hà Đông, Hà Nội"
368,NaN,"Bạch Mai, Bạch Mai, Hai Bà Trưng, Hà Nội"
397,Phường Mỹ Đình 2,"Mỹ Đình 2, Nam Từ Liêm, Hà Nội"
...,...,...
18310,Phường Dương Nội,"Dương Nội, Hà Đông, Hà Nội"
18326,NaN,"Ngõ 21 chùa bụt mọc, Phú Diễn, Phú Diễn, Bắc T..."
18371,Xã Bình Chánh,"Bình Chánh, Bình Chánh, Hồ Chí Minh"
18429,NaN,"Định Công, Định Công, Hoàng Mai, Hà Nội"


- 176: Không điền nổi
- 703: Phường Ba: Bạch Mai, Hai Bà Trưng, Hà Nội ???

__Correct values__: 306, 397, 470, 547, 833, 964, 966, 1103, 1606, 1657, 1663, 1694, 2184, 2414, 2417, 2551, 

__NaN values__
- 305: Nguyễn Viết Xuân Hòa Minh Liên Chiểu
- 368: Bạch Mai, Bạch Mai, Hai Bà Trưng, Hà Nội
- 474: Nguyễn Sơn, Bồ Đề, Long Biên, Hà Nội
- 564: Đặng Thùy Trâm, 13, Bình Thạnh, Hồ Chí Minh
- 619: Thạch Cầu, Long Biên, Long Biên, Hà Nội
- 706: Liên Mạc, Thượng Cát, Bắc Từ Liêm, Hà Nội
- 772: Trịnh Văn Bô, Xuân Phương, Nam Từ Liêm, Hà Nội
- 827: Đông Tác, Kim Liên, Đống Đa, Hà Nội
- 828: Tên Lửa, Bình Trị Đông B, Bình Tân, Hồ Chí Minh
- 1042: Hương lộ 2, Bình Trị Đông, Bình Tân, Hồ Chí Minh
- 1105: Nguyễn Trãi, Nguyễn Trãi, Thượng Đình, Thanh Xuân, Hà Nội
- 1169: Tên Lửa, Bình Trị Đông B, Bình Tân, Hồ Chí Minh
- 1283: Trịnh Như Khuê, Bình Chánh, Bình Chánh, Hồ Chí Minh
- 1727: Nguyễn Văn Bứa, Xuân Thới Sơn, Hóc Môn, Hồ Chí Minh
- 1908: Xuân Đỉnh, Xuân Đỉnh, Bắc Từ Liêm, Hà Nội
- 1994: Cống Quỳnh, Nguyễn Cư Trinh, Quận 1, Hồ Chí Minh
- 2052: Vạn Kiếp, 3, Bình Thạnh, Hồ Chí Minh
- 2111: 27, 16, Gò Vấp, Hồ Chí Minh
- 2121: Cầu Giấy, Quan Hoa, Cầu Giấy, Hà Nội
- 2127: Đặng Thùy Trâm, 13, Bình Thạnh, Hồ Chí Minh
- 2345: 6, Yên Nghĩa, Hà Đông, Hà Nội
- 2399: Khu dân cư Hòn Rớ 1, Phước Đồng, Nha Trang, Khánh Hòa
- 2430: Đại La, Đồng Tâm, Hai Bà Trưng, Hà Nội
- 2448: 427, Liên Phương, Thường Tín, Hà Nội
- 2457: FPT City Đà Nẵng, Võ Chí Công, Hòa Hải, Ngũ Hành Sơn, Đà Nẵng
- 2460: Lê Hồng Phong, 1, Quận 10, Hồ Chí Minh
- 2537: Cầu Giấy, Quan Hoa, Cầu Giấy, Hà Nội
- 2538: Nguyễn Hữu Dật, Tây Thạnh, Tân Phú, Hồ Chí Minh
- 2558, 2578, 2580, 2598, 2634 3263, 3283, 3313, 3450,3661, 3740, 3804, 4036, 4067, 4094, 

In [26]:
nan_ward = cleaned[cleaned['Xã/Phường/Thị trấn'].isna()][['Tỉnh/Thành phố', 'Thành phố/Quận/Huyện/Thị xã','Xã/Phường/Thị trấn', 'short_address', 'description_x']]

In [27]:
nan_ward

,Tỉnh/Thành phố,Thành phố/Quận/Huyện/Thị xã,Xã/Phường/Thị trấn,short_address,description_x
176,Thành phố Hồ Chí Minh,Quận Tân Bình,NaN,Nhà 2 mặt tiền hẻm xe tải thông hướng Âu Cơ Tr...,"Nhà riêng 2 mặt tiền, cực chất ở Trường Chinh,..."
305,Thành phố Đà Nẵng,Quận Liên Chiểu,NaN,Nhà 3 tầng Nguyễn Viết Xuân Hòa Minh Liên Chiể...,Chính chủ gửi bán nhà 3 tầng MT đường Nguyễn V...
368,Thành phố Hà Nội,Quận Hai Bà Trưng,NaN,"Bạch Mai, Bạch Mai, Hai Bà Trưng, Hà Nội",Nhà riêng Bạch Mai dân xây khung cột bê tông c...
474,Thành phố Hà Nội,Quận Long Biên,NaN,"Nguyễn Sơn, Bồ Đề, Long Biên, Hà Nội","Vị trí: Nguyễn Sơn, Long Biên ngõ thông, vài b..."
564,Thành phố Hồ Chí Minh,Quận Bình Thạnh,NaN,"Đặng Thùy Trâm, 13, Bình Thạnh, Hồ Chí Minh","- Cần bán nhà riêng 3 tầng, 4PN + 5WC + bếp, p..."
...,...,...,...,...,...
17985,Thành phố Hồ Chí Minh,Huyện Nhà Bè,NaN,"Huỳnh Tấn Phát, Nhà Bè, Nhà Bè, Hồ Chí Minh","Chủ gấp bán giảm giá từ 6,1 tỷ còn 5,6 tỷ bán ..."
18172,Thành phố Hà Nội,Huyện Hoài Đức,NaN,"32, Di Trạch, Hoài Đức, Hà Nội","BÁN NHÀ GIÁP NHỔN_66M2X5T, GẦN VÀNH ĐAI 3,5, K..."
18229,Thành phố Hà Nội,Quận Long Biên,NaN,"Phúc Lợi, Phúc Lợi, Long Biên, Hà Nội",SIÊU HIẾM HƠN 3 TỶ PHÚC LỢI CẠNH VINHOMES 33M2...
18326,Thành phố Hà Nội,Quận Bắc Từ Liêm,NaN,"Ngõ 21 chùa bụt mọc, Phú Diễn, Phú Diễn, Bắc T...","Chính chủ e cần bán nhà Phú Diễn, logoc 2 thoáng"


In [28]:
import sqlite3

def load_data(provinces_sql_path: str, districts_sql_path: str, wards_sql_path: str, streets_sql_path: str):
    try:
        conn = sqlite3.connect(":memory:")
        conn.execute("CREATE TABLE provinces (name TEXT, code TEXT, status TEXT);")
        conn.execute("CREATE TABLE districts (name TEXT, code TEXT, province_code TEXT, status TEXT);")
        conn.execute("CREATE TABLE wards (name TEXT, code TEXT, district_code TEXT, status TEXT);")
        conn.execute("CREATE TABLE streets (name TEXT, code TEXT, district_code TEXT, status TEXT);")

        with open(provinces_sql_path, "r", encoding="utf-8") as f:
            conn.executescript(f.read())

        with open(districts_sql_path, "r", encoding="utf-8") as f:
            dis_cleaned = f.read().replace("\\'", "''")
            conn.executescript(dis_cleaned)
        
        with open(wards_sql_path, "r", encoding="utf-8") as f:
            ward_cleaned = f.read().replace("\\'", "''")
            conn.executescript(ward_cleaned)

        with open(streets_sql_path, "r", encoding="utf-8") as f:
            street_cleaned = f.read().replace("'\\'", "''")
            street_cleaned = f.read().replace("'\'", "''")
            conn.executescript(street_cleaned)

        provinces_df = pd.read_sql_query("SELECT * FROM provinces", conn)
        districts_df = pd.read_sql_query("SELECT * FROM districts", conn)


        wards_df = pd.read_sql_query("""
            SELECT w.name AS ward_name, w.code AS ward_code, w.status AS ward_status,
                   d.name AS district_name, d.code AS district_code,
                   p.name AS province_name, p.code AS province_code
            FROM wards w
            JOIN districts d ON w.district_code = d.code
            JOIN provinces p ON d.province_code = p.code
        """, conn)

        # Join để street có thêm district + province
        streets_df = pd.read_sql_query("""
            SELECT s.name AS street_name, s.code AS street_code, s.status AS street_status,
                   d.name AS district_name, d.code AS district_code,
                   p.name AS province_name, p.code AS province_code
            FROM streets s
            JOIN districts d ON s.district_code = d.code
            JOIN provinces p ON d.province_code = p.code
        """, conn)

    except (FileNotFoundError, sqlite3.Error) as e:
        print(f"Error: Could not load administrative data. Details: {e}")
        raise
    finally:
        if 'conn' in locals():
            conn.close()

    # Build reverse lookup maps
    reverse_province_map = {
        prov.replace("Thành phố ", "").replace("Tỉnh ", ""): prov
        for prov in provinces_df['name'].unique()
    }
    reverse_province_map['Bà Rịa Vũng Tàu'] = 'Tỉnh Bà Rịa - Vũng Tàu'

    reverse_district_map = {
        dis.replace("Thành phố ", "").replace("Thành Phố ", "").replace("Huyện ", "")
            .replace("Quận ", "").replace("Thị xã ", ""): dis
        for dis in districts_df['name'].unique()
    }

    return [reverse_province_map, reverse_district_map, wards_df, streets_df]

province, district, ward, street = load_data("Dữ liệu địa giới hành chính/provinces_20250225_2.sql", 
                                             "Dữ liệu địa giới hành chính/districts_20250225_2.sql",
                                             "Dữ liệu địa giới hành chính/wards_20250225_2.sql",
                                             "Dữ liệu địa giới hành chính/streets_20250225_2.sql")

In [29]:
import unicodedata
import re

nan_ward['short_add_list'] = nan_ward['short_address'].apply(lambda x: x.split(','))

def norm_text(text):
    if isinstance(text, str):
        # 1. Normalize to composed form (fix precomposed/decomposed)
        text = unicodedata.normalize("NFC", text)
        # 2. Replace non-breaking and zero-width spaces with normal space
        text = text.replace("\u00A0", " ").replace("\u200B", "")
        # 3. Lowercase (optional, for case-insensitive matching)
        text = text.lower()
        # 4. Collapse multiple spaces/commas
        text = re.sub(r"\s*,\s*", ", ", text)   # tidy commas
        text = re.sub(r"\s+", " ", text)        # collapse spaces
        # 5. Strip
        return text.strip()
    return text

def clean_ward(row):
    if isinstance(row['Xã/Phường/Thị trấn'], str):
        return row['Xã/Phường/Thị trấn']
    short_add_list = row['short_address'].split(',')
    quanhuyen = row['Thành phố/Quận/Huyện/Thị xã']
    ward_names = ward[ward['district_name'] == quanhuyen]['ward_name'].unique()
    if len(short_add_list) >= 4:
        for i in ward_names:
            if norm_text(short_add_list[-3]) in norm_text(i):
                return unicodedata.normalize('NFD', i).strip()
    return None
        
cleaned['Xã/Phường/Thị trấn'] = cleaned.apply(clean_ward, axis=1)
cleaned['Xã/Phường/Thị trấn'].isna().sum()

np.int64(42)

In [30]:
import unicodedata
import re

def normalize_text(text):
    if not isinstance(text, str):
        return text
    # Normalize Unicode (NFC is safer for Vietnamese)
    text = unicodedata.normalize("NFC", text)
    # Remove zero-width and non-breaking spaces
    text = re.sub(r"[\u200B-\u200D\uFEFF\u00A0]", " ", text)
    # Strip leading/trailing spaces
    return text.strip()
ward_copy = ward.copy()
ward_copy["ward_name"] = ward_copy["ward_name"].apply(normalize_text)


In [31]:
vals = ward_copy[ward_copy['ward_name'].str.contains("Bình Trị Đông B")]['ward_name'].unique()
for v in vals:
    print(repr(v), [ord(ch) for ch in v])

'Phường Bình Trị Đông B' [80, 104, 432, 7901, 110, 103, 32, 66, 236, 110, 104, 32, 84, 114, 7883, 32, 272, 244, 110, 103, 32, 66]


In [32]:
val = cleaned.loc[1169]['short_address'].strip().split(',')[-3]
print(repr(v), [ord(ch) for ch in val])


'Phường Bình Trị Đông B' [32, 66, 236, 110, 104, 32, 84, 114, 7883, 32, 272, 244, 110, 103, 32, 66]


In [33]:
cleaned['Xã/Phường/Thị trấn'].unique()

array(['Phường Phương Sơn', 'Phường Tây Mỗ', 'Phường 5',
       'Phường Quan Hoa', 'Phường Tân Tạo', 'Phường 13', 'Xã Phước Lộc',
       'Xã An Khánh', 'Phường Bến Thành', 'Phường Phú Hữu', 'Xã Tân Kiên',
       'Phường Đại Mỗ', 'Phường Láng Hạ', 'Phường Ngọc Hiệp', 'Phường 15',
       'Phường Láng Hạ', 'Phường 9', 'Phường Hiệp Bình Chánh',
       'Phường Thanh Lương', 'Phường 7', 'Phường Nghĩa Đô', 'Phường 14',
       'Phường Hoàng Văn Thụ', 'Phường 1', 'Thị trấn Tân Túc',
       'Phường Hà Cầu', 'Phường 3', 'Phường Vĩnh Hải',
       'Phường Trung Liệt', 'Phường Thạch Bàn', 'Phường Phương Canh',
       'Phường 11', 'Phường 8', 'Phường Bình Trị Đông', 'Phường 25',
       'Phường Xuân La', 'Phường 19', 'Phường 17', 'Phường 6',
       'Phường Trần Hưng Đạo', 'Phường 12', 'Phường Nghĩa Tân',
       'Phường Ngọc Thụy', 'Phường Ô Chợ Dừa', 'Xã Hiệp Phước',
       'Phường Hiệp Thành', 'Phường Tăng Nhơn Phú A', 'Phường Phú Mỹ',
       'Phường Vĩnh Hòa', 'Phường Bình Trưng Đông',
      

### Đang bị vấn đề ở đây, không biết làm sao mà xét short_add_list[-3] in ward_name không đúng

In [34]:
pd.set_option('display.max_colwidth', None)
cleaned[cleaned['Xã/Phường/Thị trấn'].isna()][['short_address', 'Xã/Phường/Thị trấn']]

,short_address,Xã/Phường/Thị trấn
176,Nhà 2 mặt tiền hẻm xe tải thông hướng Âu Cơ Trường Chinh,None
305,Nhà 3 tầng Nguyễn Viết Xuân Hòa Minh Liên Chiểu Đà Nẵng,None
828,"Tên Lửa, Bình Trị Đông B, Bình Tân, Hồ Chí Minh",None
1042,"Hương lộ 2, Bình Trị Đông, Bình Tân, Hồ Chí Minh",None
1169,"Tên Lửa, Bình Trị Đông B, Bình Tân, Hồ Chí Minh",None
2457,"FPT City Đà Nẵng, Võ Chí Công, Hòa Hải, Ngũ Hành Sơn, Đà Nẵng",None
2538,"Nguyễn Hữu Dật, Tây Thạnh, Tân Phú, Hồ Chí Minh",None
4036,"Lê Tấn Bê, An Lạc, Bình Tân, Hồ Chí Minh",None
4640,"Đường Mậu Lương, TDP12 Phường Kiến Hưng, Hà Đông, Hà Nội",None
4897,"Bằng Liệt, Hoàng Liệt, Hoàng Mai, Hà Nội",None
